In [1]:
import os
import warnings

import folium
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
from shapely.geometry import Point

from network_wrangler import RoadwayNetwork
from network_wrangler import ProjectCard

%config IPCompleter.greedy=True
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
STPAUL_DIR = os.path.join(os.getcwd(),'../','example','stpaul')
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")

In [51]:
net = RoadwayNetwork.read(link_file= STPAUL_LINK_FILE, node_file=STPAUL_NODE_FILE, shape_file=STPAUL_SHAPE_FILE, fast=True)

In [52]:
easy_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        ],
     'A':{'osmNodeId': '187899923'}, # start searching for segments at A
     'B':{'osmNodeId': '187865924'}, # end at B
}

longer_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        ],
     'A':{'osmNodeId': '187899923'}, # start searching for segments at A
     'B':{'osmNodeId': '187942339'}, # end at B
}
##TODO
multi_criteria_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        {'OSMlanes': [1,2]}
         ],
     'A':{'osmNodeId': '187899923'}, # start searching for segments at A
     'B':{'osmNodeId': '187942339'}, # end at B
}

In [54]:
RoadwayNetwork.MAX_SEARCH_BREADTH = 30
net.select_roadway_features(easy_selection)
net.select_roadway_features(longer_selection)

easy_query = ProjectCard.build_link_selection_query(easy_selection)
A_id, B_id = net.orig_dest_nodes_foreign_key(easy_selection)
easy_key = (easy_query, A_id, B_id)
longer_query = ProjectCard.build_link_selection_query(longer_selection)
A_id, B_id = net.orig_dest_nodes_foreign_key(longer_selection)
longer_key = (longer_query, A_id, B_id)

In [25]:
G = net.selections[easy_key]['graph']
#G.edges
#nx.get_edge_attributes(G,'weight')
G.nodes['524837590']
#ox.plot_graph(G, fig_height=10, fig_width=16)

{'id': '',
 'isBikeNode': 1,
 'isDriveNode': 1,
 'isTranNode': 0,
 'isWalkNode': 1,
 'travelModelId': 4758,
 'geometry': <shapely.geometry.point.Point at 0xa30296f28>,
 'x': -93.0743811,
 'y': 44.95603930000001}

In [57]:
class selection_map:
    
    def __init__(self, net, selection={}):
        self.net = net
        self.selection = selection
        self.query = ProjectCard.build_link_selection_query(selection)
        
        self.selection_key = net.build_selection_key(self.selection)
        self.sp_found = self.net.select_roadway_features(selection)
        
        self.G     = self.net.selections[self.selection_key]['graph']
        self.links = self.net.selections[self.query]['links']
        self.A_name = self.selection_key[1]
        self.B_name = self.selection_key[2]
        
        self.A     = self.G.nodes[self.A_name]
        self.B     = self.G.nodes[self.B_name]
        
        if self.sp_found:
            self.sp_route = self.net.selections[self.selection_key]['route']        
            self.route_map = ox.plot_route_folium(G, self.sp_route, tiles='cartodbdark_matter')
    
    def layout(self):
        from ipywidgets import HBox, VBox, Output
        from IPython.display import display, HTML
        srcdoc = self.folium_route()._repr_html_().replace('"', '&quot;')
        html     = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: 400px;'
                 'border: none"></iframe>'.format(srcdoc))
        
        return VBox(display(self.links), display(html), display(self.query))
    
    def folium_node(self, node, node_name, color='white'):
        node_marker = folium.Marker(location=[node['y'],node['x']], 
                                    icon=folium.Icon(color=color), 
                                    tooltip=node_name,
                                   )
        return node_marker
    
    def folium_route(self):
        m = ox.plot_route_folium(self.G, self.sp_route, route_map=self.folium_graph(),
                          route_color='white', route_width=15, route_opacity=.5)
        return m
    
    def folium_graph(self):
        m = ox.plot_graph_folium(self.G, 
                                 popup_attribute='weight', 
                                 edge_color='blue',
                                 edge_width=2, 
                                 tiles='cartodbdark_matter')
        self.folium_node(self.A, self.A_name, color="green").add_to(m)
        self.folium_node(self.B, self.B_name, color="pink").add_to(m)
        
        return m
    
    def candidate_graph(self):
        
        m = ox.plot_graph_folium(self.G, 
                                 popup_attribute='weight', 
                                 edge_color='blue',
                                 edge_width=.2, 
                                 tiles='cartodbdark_matter')
        
        links = ox.save_load.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
        colors = ox.plot.get_colors(10)
        colors_d = {c:value for c, value in enumerate(colors, 0)}
        links['color'] = links['i'].map(colors_d)
        #print(links[['i','color']])
        
        for _, row in links.iterrows():
            pl = ox.plot.make_folium_polyline(edge=row, edge_color=row['color'], edge_width=5,
                                      edge_opacity=1.0, popup_attribute='weight')
            pl.add_to(m)
        self.folium_node(self.A, self.A_name, color="green").add_to(m)
        self.folium_node(self.B, self.B_name, color="pink").add_to(m)
        
        return m

In [58]:
sel = selection_map(net, selection=longer_selection)
#sel.folium_graph()
#sel.folium_route()
#sel.layout()
sel.candidate_graph()

TypeError: build_selection_key() missing 1 required positional argument: 'selection_dict'